In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import os
import gc
import cv2
import math
import copy
import time
import random
import glob
from matplotlib import pyplot as plt

# For data manipulation
import numpy as np
import pandas as pd

# Pytorch Imports
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torch.cuda import amp
import torchvision

# Utils
import joblib
from tqdm import tqdm
from collections import defaultdict

# Sklearn Imports
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold

# For Image Models
import timm

# Albumentations for augmentations
import albumentations as A
from albumentations.pytorch import ToTensorV2
from albumentations.core.transforms_interface import ImageOnlyTransform

# For colored terminal text
from colorama import Fore, Back, Style
b_ = Fore.BLUE
sr_ = Style.RESET_ALL

import warnings
warnings.filterwarnings("ignore")

# For descriptive error messages
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [2]:
CONFIG = {
    "seed": 2455,
    "img_size": 512,
    "model_name": "efficientnetv2_rw_s", #"tf_efficientnetv2_s_in21ft1k",
    "num_classes": 5,
    "valid_batch_size": 32,
    "device": torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
}

In [9]:
ROOT_DIR = '/kaggle/input/UBC-OCEAN'
TEST_DIR = '/kaggle/input/UBC-OCEAN/test_thumbnails'
ALT_TEST_DIR = '/kaggle/input/UBC-OCEAN/test_images'
# TEST_DIR = '/kaggle/input/UBC-OCEAN/train_thumbnails'
# ALT_TEST_DIR = '/kaggle/input/UBC-OCEAN/train_images'
LABEL_ENCODER_BIN = ['CC', 'EC', 'HGSC', 'LGSC', 'MC']
BEST_WEIGHT = "/kaggle/input/effiv2/4E-val0.5517857142857143-efficientnetv2_rw_s.pth"

In [10]:
def set_seed(seed=42):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)
    
set_seed(CONFIG['seed'])

In [11]:
def get_test_file_path(image_id):
    if os.path.exists(f"{TEST_DIR}/{image_id}_thumbnail.png"):
        return f"{TEST_DIR}/{image_id}_thumbnail.png"
    else:
        return f"{ALT_TEST_DIR}/{image_id}.png"

In [12]:
df = pd.read_csv(f"{ROOT_DIR}/test.csv")
# df = pd.read_csv(f"{ROOT_DIR}/train.csv")
df['file_path'] = df['image_id'].apply(get_test_file_path)
df['label'] = 0 # dummy
df

In [13]:
df_sub = pd.read_csv(f"{ROOT_DIR}/sample_submission.csv")
df_sub

In [14]:
# encoder = joblib.load( LABEL_ENCODER_BIN )
encoder = LABEL_ENCODER_BIN

In [ ]:
class UBCDataset(Dataset):
    def __init__(self, df, transforms=None):
        self.df = df
        self.file_names = df['file_path'].values
        self.labels = df['label'].values
        self.transforms = transforms
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        img_path = self.file_names[index]
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        label = self.labels[index]
        
        if self.transforms:
            img = self.transforms(image=img)["image"]
            
        return {
            'image': img,
            'label': torch.tensor(label, dtype=torch.long)
        }

In [ ]:
class BackgroundRemove(ImageOnlyTransform):
    def __init__(self,
                 threshold=215, 
                 always_apply: bool = False, p: float = 0.5):
        super(BackgroundRemove, self).__init__(always_apply, p)
        self.threshold = threshold
    
    def apply(self, img, **params):
        return self.remove_background(img)
    
    def remove_background(self, img):
        img[np.where((img > [self.threshold, self.threshold, self.threshold]).all(axis = 2))] = [0,0,0]
        return img
    
    def get_transform_init_args_names(self):
        return ("threshold",)
    

In [ ]:
# class GeM(nn.Module):
#     def __init__(self, p=3, eps=1e-6):
#         super(GeM, self).__init__()
#         self.p = nn.Parameter(torch.ones(1)*p)
#         self.eps = eps

#     def forward(self, x):
#         return self.gem(x, p=self.p, eps=self.eps)
        
#     def gem(self, x, p=3, eps=1e-6):
#         return F.avg_pool2d(x.clamp(min=eps).pow(p), (x.size(-2), x.size(-1))).pow(1./p)
        
#     def __repr__(self):
#         return self.__class__.__name__ + \
#                 '(' + 'p=' + '{:.4f}'.format(self.p.data.tolist()[0]) + \
#                 ', ' + 'eps=' + str(self.eps) + ')'

In [ ]:
# class UBCModel(nn.Module):
#     def __init__(self, model_name, num_classes, pretrained=False, checkpoint_path=None):
#         super(UBCModel, self).__init__()
#         self.model = timm.create_model('tf_efficientnetv2_s_in21ft1k', pretrained=pretrained)

#         in_features = self.model.classifier.in_features
#         self.model.classifier = nn.Identity()
#         self.model.global_pool = nn.Identity()
#         self.pooling = GeM()
#         self.linear = nn.Linear(in_features, num_classes)
#         self.softmax = nn.Softmax(dim=1)

#     def forward(self, images):
#         features = self.model(images)
#         pooled_features = self.pooling(features).flatten(1)
#         output = self.linear(pooled_features)
#         return output

class BaseModel(nn.Module) :
    def __init__(self, model_name, num_classes) -> None:
        super(BaseModel, self).__init__()
        self.backbone = timm.create_model(model_name=model_name, pretrained=False)

        self.model = nn.Sequential(
            self.backbone,
            nn.Linear(1000, num_classes)
        )
        
    def forward(self, x) :
        return self.model(x)
    
model = BaseModel(CONFIG['model_name'], CONFIG['num_classes'])
model.load_state_dict(torch.load( BEST_WEIGHT )['model_state_dict'])
model.to(CONFIG['device']);

In [ ]:
data_transforms = {
    "valid": A.Compose([
        A.Resize(CONFIG['img_size'], CONFIG['img_size']),
        BackgroundRemove(always_apply=True), ## 0.25 점수 제출하고 추가
        A.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
        ToTensorV2()], p=1.)
}


test_dataset = UBCDataset(df, transforms=data_transforms["valid"])
test_loader = DataLoader(test_dataset, batch_size=CONFIG['valid_batch_size'], 
                          num_workers=2, shuffle=False, pin_memory=True)

In [ ]:
preds = []
with torch.no_grad():
    bar = tqdm(enumerate(test_loader), total=len(test_loader))
    for step, data in bar:        
        images = data['image'].to(CONFIG["device"], dtype=torch.float)        
        batch_size = images.size(0)
#         outputs = model(images)
        predicted = model(images).argmax(1).detach().cpu().numpy().tolist()
        preds.append(predicted)
preds = np.concatenate(preds).flatten()
# pred_labels = encoder.inverse_transform( preds )

In [ ]:
pred_labels = [encoder[p] for p in preds]

In [ ]:
df_sub["label"] = pred_labels
df_sub.to_csv("submission.csv", index=False)